In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)

from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from datetime import datetime
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
)
from amftrack.util.dbx import read_saved_dropbox_state, get_dropbox_folders
import sys
import os

from amftrack.util.sys import get_dirname, temp_path
import pandas as pd
import ast
from scipy import sparse
import scipy.io as sio
import cv2
import imageio.v2 as imageio
import numpy as np
import scipy.sparse
import os
from time import time
from amftrack.pipeline.functions.image_processing.extract_skel import (
    extract_skel_new_prince,
    run_back_sub,
    bowler_hat,
)

from amftrack.util.sys import get_dates_datetime, get_dirname
import shutil
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
    Edge,
    Node,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    reconstruct_image_from_general,
    plot_full,
    plot_edge_color_value,
)
from matplotlib import cm

In [ ]:
plates = [
    "938_20220325",
    "1045_20220504",
    "1048_20220506",
    "1076_20220511",
    # "241_20230403",
    "247_20230422",
    "24_20220426",
]

In [ ]:
directory_targ = os.path.join(directory_scratch, "stitch_temp2") + "/"
# directory_targ = directory_project

update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

In [ ]:
folders = all_folders.loc[all_folders["unique_id"] == "24_20220426"]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_width.p"] == True]

In [ ]:
folders.loc[folders["strain"] == "'A5'"]

In [ ]:
folders = folders.sort_values(by="datetime")

exp = Experiment(directory_targ)
i = 45
exp.load(folders.iloc[i : i + 1], suffix="_width")
for t in range(exp.ts):
    exp.load_tile_information(t)

In [ ]:
timestep = {
    "938_20220325": 35,
    "1045_20220504": 45,
    "1048_20220506": 45,
    "1076_20220511": 65,
    "247_20230422": 45,
    "24_20220426": 60,
}
results = {}
for plate in plates:
    folders = all_folders.loc[all_folders["unique_id"] == plate]
    folders = folders.loc[folders["/Analysis/nx_graph_pruned_width.p"] == True]
    folders = folders.sort_values(by="datetime")

    exp = Experiment(directory_targ)
    i = 45
    exp.load(folders.iloc[i : i + 1], suffix="_width")
    for t in range(exp.ts):
        exp.load_tile_information(t)
    edges = get_all_edges(exp, t)
    lengths = [edge.length_um(t) for edge in edges]
    widths = [edge.width(t) for edge in edges]
    results[plate] = edges, lengths, widths

In [ ]:
select = all_folders.loc[all_folders["unique_id"].isin(plates)]
select = select.loc[select["/Analysis/nx_graph_pruned_width.p"] == True]

strains = select["strain"].unique()
plate_strain = {}
for plate in plates:
    folders = all_folders.loc[all_folders["unique_id"] == plate]
    strain = folders["strain"].value_counts().idxmax()
    plate_strain[plate] = strain
strain_plate = {}
for strain in plate_strain.values():
    strain_plate[strain] = []
    for key in plate_strain.keys():
        if plate_strain[key] == strain:
            strain_plate[strain].append(key)
result_strain = {}
for strain in strains:
    plates_strain = strain_plate[strain]
    full_edges, full_lengths, full_widths = [], [], []
    for plate in plates_strain:
        edges, lengths, widths = results[plate]
        full_edges += edges
        full_lengths += lengths
        full_widths += widths
    result_strain[strain] = full_edges, full_lengths, full_widths

In [ ]:
fig, ax = plt.subplots()
for plate in plates:
    edges, lengths, widths = results[plate]
    ax.hist(widths, 30, weights=lengths, alpha=0.2, density=True)
    ax.set_xlim(0, 15)
# ax.set_yscale("log")

In [ ]:
fig, ax = plt.subplots()
for strain in strains:
    edges, lengths, widths = result_strain[strain]
    ax.hist(
        np.array(widths) / 2,
        30,
        weights=lengths,
        alpha=0.2,
        density=True,
        label=f"{strain}"
        + " : $r_{eff} =$"
        + str(
            round(np.sqrt(np.average(np.array(widths) ** 2, weights=lengths)) / 2, 1)
        ),
    )
    ax.set_xlim(0, 7.5)
# ax.set_yscale("log")
ax.set_xlabel("hyphal radius ($\mu m$)")
ax.set_ylabel("length weighted probability")

ax.legend()

In [ ]:
strains

In [ ]:
vmax = 9
vmin = 3
region = None
nodes = get_all_nodes(exp, t)
plot_edge_color_value(
    exp,
    t,
    lambda edge: edge.width(t),
    region=region,
    # nodes = nodes,
    cmap=cm.get_cmap("viridis", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=True,
    dilation=10,
    figsize=(16, 12),
    alpha=0.3,
    downsizing=5,
)

In [ ]:
vmax = 9
vmin = 3
region = None
nodes = get_all_nodes(exp, t)
plot_edge_color_value(
    exp,
    t,
    lambda edge: edge.width(t),
    region=region,
    # nodes = nodes,
    cmap=cm.get_cmap("viridis", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=True,
    dilation=4,
    figsize=(16, 12),
    alpha=0.3,
    downsizing=5,
)

In [ ]:
vmax = 9
vmin = 3
region = [[3800 * 5, 4000 * 5], [3600 * 5, 3700 * 5]]
nodes = get_all_nodes(exp, t)
plot_edge_color_value(
    exp,
    t,
    lambda edge: edge.width(t),
    region=region,
    nodes=nodes,
    cmap=cm.get_cmap("viridis", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=True,
    dilation=4,
    figsize=(16, 12),
    alpha=0.3,
    downsizing=1,
)

In [ ]:
edges1 = [
    (30162, 29280),
    (29962, 29194),
    (30139, 31086),
    (31459, 32379),
    (32614, 32260),
    (32260, 31847),
]

In [ ]:
edges1_obj = [Edge(Node(edge[0], exp), Node(edge[1], exp), exp) for edge in edges1]

In [ ]:
edges1_obj[-1].width(t)

In [ ]:
19.7 * 1.725 / 12.5

In [ ]:
profile, _, __ = extract_section_profiles_for_edge_exp(
    exp,
    t,
    edge,
    resolution=resolution,
    offset=offset,
    step=step,
    target_length=target_length,
)

In [ ]:
vmax = 9
vmin = 3
region = [[3800 * 5, 4000 * 5], [3600 * 5, 3700 * 5]]
nodes = get_all_nodes(exp, t)
plot_edge_color_value(
    exp,
    t,
    lambda edge: edge.width(t),
    region=region,
    nodes=nodes,
    cmap=cm.get_cmap("viridis", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=True,
    dilation=2,
    figsize=(16, 12),
    alpha=0,
    downsizing=1,
)

In [ ]:
vmax = 12
vmin = 0
plot_edge_color_value(
    exp,
    t,
    lambda edge: edge.width(t),
    cmap=cm.get_cmap("seismic", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=10,
    figsize=(12, 8),
)

In [ ]:
edges = get_all_edges(exp, t)
lengths = [edge.length_um(t) for edge in edges]
widths = [edge.width(t) for edge in edges]
fig, ax = plt.subplots()
ax.hist(widths, 30, weights=lengths)
ax.set_xlim(0, 15)

In [ ]:
edges = get_all_edges(exp, t)
lengths = [edge.length_um(t) for edge in edges]
widths = [edge.width(t) for edge in edges]
fig, ax = plt.subplots()
ax.hist(widths, 30, weights=lengths)
ax.set_xlim(0, 15)

In [ ]:
edges = get_all_edges(exp, t)
lengths = [edge.length_um(t) for edge in edges]
widths = [edge.width(t) for edge in edges]
fig, ax = plt.subplots()
ax.hist(widths, 20, weights=lengths)

In [ ]:
np.average(widths, weights=lengths)

In [ ]:
np.average(widths, weights=lengths)

In [ ]:
np.average(widths, weights=lengths)

In [ ]:
np.average(widths, weights=lengths)